In [21]:
import os
import requests
from newsapi import NewsApiClient

from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import JsonOutputParser

from langgraph.graph import StateGraph, START, END

from typing import TypedDict, Literal

In [22]:
load_dotenv()

#Create State for LangGraph
class NewsState(TypedDict):
    
    topic: str
    sentiment: str
    summarize: str
    news: str
    fetchCount: int
    concise: str

In [23]:
#Initialize NewsAPI model
news_api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=news_api_key)

#Initialize LLM Model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


In [24]:

#Fetch News from NewsApi
def fetch_news(state: NewsState):   
    try:
        topic = state['topic']
        # /v2/top-headlines
        response = newsapi.get_everything(q=topic,                          
                                          language='en',
                                          page=1,
                                          page_size=10)
        
        if response['status'] == 'ok' and response['totalResults'] > 0:       
            
            fetchCount = state['fetchCount']
            state['fetchCount'] = fetchCount + 1
            docs = ""
            
            articles = response.get('articles', [])
            for idx, article in enumerate(articles):
                # docs.append(
                #     Document(
                #         page_content = f"""
                #             {article['content']}
                #         """,
                #         metadata={"source": article['author'], "title": article['title'], "published_at": article['publishedAt']}
                #     )
                # )
                docs += f"""
                
                Author: {article['author']}
                Title: {article['title']}
                Published At: {article['publishedAt']}
                Content: {article['content']}
                
                """
                
            
            # print(docs)
            state['news'] = ""
            return {'news': docs}   
        
        else:
            print("No Data Found")
            return state
        
    except Exception as e:
        print("Error while fetching news from newsapi.org api -> ", e, state)
        return state
        
#Summarize
def collect_summarize(state: NewsState):
    
    docs = state['news']
    prompt = f"I need a complete summary of the topic {state['topic']} from the following data {state['news']}"
    result = model.invoke(prompt).content
    
    # state['summarize'] = result
    return {'summarize': result}

#Find Sentiment
def find_sentiment(state: NewsState):
    
    summarize = state['summarize']
    prompt = PromptTemplate.from_template(
        "Provide a JSON object with key 'sentiment' and value should be any one of ['positive', 'negative'] for a article summary. {summarize_news}",
    )
    partial_prompt = prompt.partial(summarize_news=summarize)
    
    parser = JsonOutputParser()
    
    chain = partial_prompt | model | parser
    result = chain.invoke({})
    # state['sentiment'] = result
    
    return {'sentiment': result['sentiment']}

def checkSentimentCondition(state: NewsState) -> Literal["concise_report", "fetch_news"]:
    print(state['sentiment'])
    if state['sentiment'] == 'positive':
        print("Yes")
        return "concise_report"
    
    else:    
        if state['fetchCount'] <= 5:
            return "fetch_news"
        else:
            return "concise_report"

#Concise Report
def concise_report(state: NewsState):
    print("I'm in now Concise Report Node")
    return {'concise': 'yes'}


In [25]:
#LangGraph graph
graph = StateGraph(NewsState)

graph.add_node("fetch_news", fetch_news)
graph.add_node("collect_summarize", collect_summarize)
graph.add_node("find_sentiment", find_sentiment)
graph.add_node("concise_report", concise_report)


#Create Edges
graph.add_edge(START, "fetch_news")
graph.add_edge("fetch_news", "collect_summarize")
graph.add_edge("collect_summarize", "find_sentiment")
graph.add_conditional_edges("find_sentiment", checkSentimentCondition)
# graph.add_edge("find_sentiment", "concise_report")
# graph.add_edge("find_sentiment", "fetch_news")
graph.add_edge("concise_report", END)

In [26]:
#Compile
workflow = graph.compile()
workflow

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`